<a href="https://colab.research.google.com/github/polymoe/datascientest/blob/main/Light_03_python_rnn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Réseau de neurones récurrents </h1>
<h2 style = "text-align:center" > RNN Embedding - Classification </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

Ce notebook est destiné à pratiquer les notions évoquées dans le troisième exercice du module sur une machine plus adaptée.

Si c'est la première fois que vous utilisez colab, n'hésitez pas à jeter un coup d'oeil sur ce [notebook](https://colab.research.google.com/drive/1jXEKOk3mRYBqFWoVwJ0ZpsRJCWj46Yxt?usp=sharing).


In [6]:
## Importation des données et packages
!pip install swifter
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader punkt
!wget https://train-exo.s3.eu-west-1.amazonaws.com/677/MovieReview.csv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 33.1 MB/s 
     |████████████████████████████████| 281 kB 73.0 MB/s 
     |████████████████████████████████| 140 kB 70.9 MB/s 
  Created wheel for swifter: filename=swifter-1.1.4-py3-none-any.whl size=13932 sha256=14d8ad53bd5793a9446207ba7e4c66ce3fcac0bff408f0405d2bce820b56f3dc
  Stored in directory: /root/.cache/pip/wheels/e6/ca/b7/6b63fdb01fddb851b2b2b45c3812c6eec7a41e79e62ac99276
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
--2022-07-03 19:08:07--  https://train-exo.s3.eu-west-1.amazonaws.com/677/MovieReview.csv
Resolving train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)... 52.218.91.0
Connecting to train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)|52.218.91.0|:443... connected.

Explication activation GPU et utilité

* Exécuter la cellule suivante pour vérifier que le GPU soit bien activé.

In [3]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0



## Jeu de données

> L'objectif de cet exercice est d'ajouter la dimension de séquentialité dans un problème d'analyse de sentiment. Le jeu de données contient 25000 critiques de films provenant du site IMDB labélisées en positives/négatives.

* Importer le module **`pandas`** sous le nom **`pd`**.


* Charger le fichier ***'MovieReview.csv'*** sous le nom **`df`**.


* Remplacer les valeurs 'Negative' et 'Positive' de la colonne "sentiment" par 0 et 1.


* Afficher les 5 premières lignes du dataframe.

In [ ]:
## Insérez votre code ici



In [7]:
#@title Solution
import pandas as pd
# Charger le jeu de données MovieReview.csv
df = pd.read_csv('MovieReview.csv')
# Remplacer les valeurs de la colonne sentiment par 0 ou 1.
df.sentiment = df.sentiment.replace(['Negative', 'Positive'], [0, 1])
df.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,'The Classic War of the Worlds' by Timothy Hin...
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


* Exécuter la cellule suivante pour nettoyer les données et enlever les stopwords.

In [8]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import swifter

stop_words = stopwords.words('english')

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df.review = df.review.swifter.apply(lambda x :preprocess_sentence(x))
df.head()

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

,sentiment,review
0,1,stuff going moment started listening music wat...
1,1,classic war worlds timothy hines entertaining ...
2,0,film starts manager nicholas bell giving welco...
3,0,must assumed praised film greatest filmed oper...
4,1,superbly trashy wondrously unpretentious explo...


* Séparer le jeu de données **`df.review`** et **`df.sentiment`** en un ensemble d'entraînement **`X_text_train`**, **`y_train`**, et en un ensemble de test **`X_text_test`**, **`y_test`**. Nous choisirons un rapport de 80% pour les données d'entraînements et une graine aléatoire 1234.

In [ ]:
## Insérez votre code ici



In [9]:
#@title Solution
from sklearn.model_selection import train_test_split
X_text_train, X_text_test, y_train, y_test = train_test_split(df.review, df.sentiment, test_size=0.2, random_state=1234)


## Tokenizer

* Définir un objet **`tokenizer`** à l'aide du constructeur `Tokenizer` de **`tensorflow.keras.preprocessing.text`** en précisant une limite de mots du dictionnaire de 10000.


* Mettre à jour le dictionnaire de notre **`tokenizer`** à partir de **`X_text_train`** en utilisant de la méthode `fit_on_texts`.

In [ ]:
## Insérez votre code ici



In [10]:
#@title Solution
import tensorflow as tf
# Définition du tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
# Mettre à jour le dictionnaire du tokenizer
tokenizer.fit_on_texts(X_text_train)

* Stocker le dictionnaire de correspondance entre mots et index dans la variable **`word2idx`** à l'aide de l'attribut **`word_index`** du tokenizer.


* Stocker le dictionnaire de correspondance entre index et mot dans la variable **`idx2word`** à l'aide de l'attribut **` idx2word`** du tokenizer.


* Stocker la taille du dictionnaire dans la variable **`vocab_size`** à l'aide de l'attribut **`num_words`** du tokenizer.

In [ ]:
## Insérez votre code ici



In [11]:
#@title Solution
# Définition des dictionnaires
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words


* Transformer chaque review **`X_text_train`** en une séquence d'entiers à l'aide de la méthode `texts_to_sequences`. Stocker le résultat dans la variable **X_train**.


* Faire de même pour **`X_text_test`** et stocker la liste dans **`X_test`**.

In [ ]:
## Insérez votre code ici



In [12]:
#@title Solution
X_train = tokenizer.texts_to_sequences(X_text_train)

X_test = tokenizer.texts_to_sequences(X_text_test)




* Transfomer la liste de sequence **`X_train`** en tableau numpy à l'aide de la fonction `pad_sequences`. Préciser en argument de la fonction `maxlen=500`, `padding='post'` et `truncating='post'`. Stocker le résultat dans la variable **`X_train`**.


* Faire de même pour **`X_test`**.

In [ ]:
## Insérez votre code ici



In [13]:
#@title Solution
maxlen = 500
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

## Modèle : RNN

> Le modèle sera construit de la manière suivante :
>
><img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/python_deepnlp_rnn.png' style='width:500px'>
>


* Définir un modèle `Sequential` sous le nom **`model`**.


* Ajouter une couche `Embedding` en précisant une entrée de taille 10000 et une sortie de taille 200.


* Ajouter une couche `RNN` avec une cellule `GRUCell` de 128 neurones. La couche `RNN` doit retourner une séquence. (argument **`return_sequences`**).


* Ajouter une couche `Dropout` afin d'éviter de trop de surapprentissage.


* Ajouter la couche `GlobalAveragePooling1D` afin d'appliquer notre **bag of word**.


* Ajouter une couche `Dense` avec 256 neurones et une fonction activation `relu`.


* Ajouter une couche `Dropout` afin d'éviter de trop de surapprentissage.


* Ajouter une dernière couche `Dense` avec une fonction d'activation `softmax`.


* Afficher le résumé du modèle.

In [ ]:
## Insérez votre code ici



In [14]:
#@title Solution
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, RNN, GRUCell, Dropout
embedding_dim = 200

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(RNN(GRUCell(128), return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 200)         2000000   
                                                                 
 rnn (RNN)                   (None, None, 128)         126720    
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

* Compiler le modèle avec un **optimizer** 'adam', une **fonction de perte** 'sparse_categorical_crossentropy' et une métrique 'accuracy'.


* Entraîner le modèle sur quelques epochs.

In [ ]:
## Insérez votre code ici



In [15]:
#@title Solution
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train.values,
    batch_size = 64,
    epochs=10,
    validation_data = [X_test, y_test.values])

Epoch 1/10
313/313 [==============================] - 396s 1s/step - loss: 0.4860 - accuracy: 0.7349 - val_loss: 0.3566 - val_accuracy: 0.8452
Epoch 2/10
313/313 [==============================] - 395s 1s/step - loss: 0.2218 - accuracy: 0.9146 - val_loss: 0.3003 - val_accuracy: 0.8798
Epoch 3/10
313/313 [==============================] - 393s 1s/step - loss: 0.1470 - accuracy: 0.9493 - val_loss: 0.4057 - val_accuracy: 0.8482
Epoch 4/10
313/313 [==============================] - 390s 1s/step - loss: 0.0937 - accuracy: 0.9691 - val_loss: 0.4820 - val_accuracy: 0.8772
Epoch 5/10
313/313 [==============================] - 388s 1s/step - loss: 0.0504 - accuracy: 0.9838 - val_loss: 0.4756 - val_accuracy: 0.8766
Epoch 6/10
313/313 [==============================] - 388s 1s/step - loss: 0.0357 - accuracy: 0.9888 - val_loss: 0.6444 - val_accuracy: 0.8712
Epoch 7/10
313/313 [==============================] - 390s 1s/step - loss: 0.0231 - accuracy: 0.9924 - val_loss: 0.5580 - val_accuracy: 0.8716

* Évaluer le modèle sur l'échantillon de validation.

In [ ]:
## Insérez votre code ici



In [18]:
#@title Solution
model.evaluate(X_test, y_test.values, batch_size=128)

40/40 [==============================] - 9s 214ms/step - loss: 0.8372 - accuracy: 0.8600


[0.8372064232826233, 0.8600000143051147]

## Conclusion

> La couche RNN que l'on vient d'ajouter permet de parcourir la séquence des mots encodés afin de **capturer les relations entre les mots**. Cette approche permet de **constater un gain de quelques pourcents** comparé à l'approche classique ou avec du word embedding.
>
> Mais, comme les calculs s'effectuent de manière séquentielle, cela rend les RNN globalement très lents (peu parallélisable). Les calculs en cours de développement ultérieurs passent la plupart du temps en attente des précédents en raison de la dépendance directe.